In [1]:
#!/usr/bin/env python 
import sys
import os
import time
import subprocess as sp
import itertools
import pandas as pd
## CV
import cv2
import skvideo.io
import matplotlib.pyplot as plt
#%matplotlib inline
## Model
import numpy as np
import tensorflow as tf
## Tools
import utils
## Parameters
import params ## you can modify the content of params.py

from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, ELU, Cropping2D, BatchNormalization
from keras.layers import Conv2D, SpatialDropout2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras import regularizers

## Test epoch
epoch_ids = range(1,11)
## Load model
#model = utils.get_model()

## Preprocess
def img_pre_process(img):
    """
    Processes the image and returns it
    :param img: The image to be processed
    :return: Returns the processed image
    """
    ## Chop off 1/3 from the top and cut bottom 150px(which contains the head of car)
    shape = img.shape
    img = img[int(shape[0]/3):shape[0]-150, 0:shape[1]]
    ## Resize the image
    img = cv2.resize(img, (params.FLAGS.img_w, params.FLAGS.img_h), interpolation=cv2.INTER_AREA)
    ## Return the image sized as a 4D array
    return np.resize(img, (params.FLAGS.img_w, params.FLAGS.img_h, params.FLAGS.img_c))


## Process video
def extract_frames(mode='load'):
    image_set = []
    steerings = []
    for epoch_id in epoch_ids:
        print('---------- processing video for epoch {} ----------'.format(epoch_id))
        vid_path = utils.join_dir(params.data_dir, 'epoch{:0>2}_front.mkv'.format(epoch_id))
        csv_path = "./epochs/epoch{:0>2d}_steering.csv".format(epoch_id)
        dir_name = "./epochs/epoch{:0>2d}_front_frames".format(epoch_id)
        steering = pd.read_csv(csv_path)['wheel'].tolist()
        steerings.extend(steering)
        
        if mode == 'save':
            os.mkdir(dir_name)
        frame_ids = pd.read_csv(csv_path)['ts_micro'].tolist()
        assert os.path.isfile(vid_path)
        frame_count = utils.frame_count(vid_path)
        cap = skvideo.io.vreader(vid_path) #cv2.VideoCapture(vid_path)

        machine_steering = []

        print('performing inference...')
        time_start = time.time()
        frame_count = 0
        for img in cap:
            #assert img
            ## you can modify here based on your model
            #print(img.shape)
            #cv2.imshow('frame',img)
            img = img_pre_process(img)
            if mode == 'save':
                cv2.imwrite("{}/{}.jpg".format(dir_name, frame_ids[frame_count-1]), img)
            elif mode == 'load':
                image_set.append(img)
                #steerings.append(frame_ids[frame_count-1])
            #deg = float(model.predict(img, batch_size=1))
            #machine_steering.append(deg)
            frame_count += 1

        del cap

        fps = frame_count / (time.time() - time_start)
    
        print('completed inference, total frames: {}, average fps: {} Hz'.format(frame_count, round(fps, 1)))
    return np.array(image_set), np.array(steerings)
    
        #print('performing visualization...')
        #utils.visualize(epoch_id, machine_steering, params.out_dir,
                            #verbose=True, frame_count_limit=None)
            
            

Using TensorFlow backend.


In [2]:
def load_dataset():
    image_set = []
    steerings = []
    for epoch_id in epoch_ids:
        csv_path = "./epochs/epoch{:0>2d}_steering.csv".format(epoch_id)
        dir_name = "./epochs/epoch{:0>2d}_front_frames".format(epoch_id)
        filenames = pd.read_csv(csv_path)['ts_micro'].tolist()
        steering = pd.read_csv(csv_path)['wheel'].tolist()
        steerings.extend(steering)
        #print(steerings)
        for img in filenames:
            image = cv2.imread(dir_name+str(img)+'.jpg')
            #assert image
            if image == None:
                print(img)
            image_set.append(image)
    return np.array(image_set), np.array(steerings)       
            
            
#extract_frames()
X, y = extract_frames(mode='load') #load_dataset()
print(X.shape, y.shape)
    
    

---------- processing video for epoch 1 ----------
performing inference...
completed inference, total frames: 1500, average fps: 118.9 Hz
---------- processing video for epoch 2 ----------
performing inference...
completed inference, total frames: 3900, average fps: 132.9 Hz
---------- processing video for epoch 3 ----------
performing inference...
completed inference, total frames: 2700, average fps: 125.0 Hz
---------- processing video for epoch 4 ----------
performing inference...
completed inference, total frames: 2700, average fps: 126.5 Hz
---------- processing video for epoch 5 ----------
performing inference...
completed inference, total frames: 2700, average fps: 122.5 Hz
---------- processing video for epoch 6 ----------
performing inference...
completed inference, total frames: 2700, average fps: 129.5 Hz
---------- processing video for epoch 7 ----------
performing inference...
completed inference, total frames: 2700, average fps: 130.5 Hz
---------- processing video for ep

In [10]:
table = pd.read_csv('./epochs/epoch01_steering.csv')
steering = table['wheel'].tolist()
print(steering)

[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.5, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5, -0.5, -0.5, -0.5, -0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.

In [7]:
image_shape = (params.FLAGS.img_w, params.FLAGS.img_h, 3)

model = Sequential()
model.add(Lambda(lambda x: x / 225.0 - 0.5, input_shape=image_shape))
#model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Conv2D(24, (5,5), strides=(2,2), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.5))

model.add(Conv2D(36, (5,5), strides=(2,2), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.5))

model.add(Conv2D(48, (5,5), strides=(2,2), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.5))

model.add(Conv2D(64, (3,3), strides=(1,1), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.5))

model.add(Conv2D(64, (3,3), strides=(1,1), activation=None, padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(SpatialDropout2D(0.5))

model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(100,kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('elu'))
model.add(Dropout(0.2))
model.add(Dense(50,kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('elu'))
model.add(Dropout(0.2))
model.add(Dense(10,kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

In [ ]:
filepath="./models/weights-improvement-{epoch:02d}-{val_loss:.2f}.h5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]

history_object = model.fit(X,
                           y,
                           batch_size=128,
                           validation_split=0.2,
                           epochs=1,
                           shuffle=True)
                           #callbacks=callbacks_list)

Train on 21600 samples, validate on 5400 samples
Epoch 1/1
20096/21600 [==========================>...] - ETA: 6s - loss: 39.2441